In [1]:
import pulp as pl
import numpy as np
from scipy.stats import multivariate_t

print("==============================================================================")

# Define the problem
model = pl.LpProblem("Maximize_Profit", pl.LpMaximize)

# Products, machines and days
subproducts = ['Ciasto_na_chleb_domowy_0_5kg', 'Ciasto_na_chleb_domowy_0_9kg', 'Ciasto_na_chleb_na_zakwasie']
products = ['Chleb_domowy_0_5kg', 'Chleb_domowy_0_9kg', 'Chleb_na_zakwasie']
days = ['Pon']
machines = ['Dzieza', 'Wazenie', 'Tabula']

# Income from product sales and production cost in PLN/piece
product_income = {'Chleb_domowy_0_5kg': 5.2, 'Chleb_domowy_0_9kg': 7.9, 'Chleb_na_zakwasie': 6.2}
sub_product_cost = {'Ciasto_na_chleb_domowy_0_5kg': 1.6, 'Ciasto_na_chleb_domowy_0_9kg': 2.0, 'Ciasto_na_chleb_na_zakwasie': 2.0}
product_cost = {'Chleb_domowy_0_5kg': 0.8, 'Chleb_domowy_0_9kg': 0.9, 'Chleb_na_zakwasie': 0.8}

# Time available
shifts_per_day = 2
hours_per_shift = 8
hours_per_day = shifts_per_day * hours_per_shift

max_batches = 1
large_number = 10000

production_times = {
    'Ciasto_na_chleb_domowy_0_5kg': {'Dzieza': 4.7, 'Wazenie': 0.29, 'Tabula': 0.7},
    'Ciasto_na_chleb_domowy_0_9kg': {'Dzieza': 5, 'Wazenie': 0.2, 'Tabula': 0.55},
    'Ciasto_na_chleb_na_zakwasie': {'Dzieza': 8.6, 'Wazenie': 0.22, 'Tabula': 0.6}
}
batch_size = {
    "Ciasto_na_chleb_domowy_0_5kg": 240,
    "Ciasto_na_chleb_domowy_0_9kg": 130,
    "Ciasto_na_chleb_na_zakwasie": 200,
    "Chleb_domowy_0_5kg": 240,
    "Chleb_domowy_0_9kg": 130,
    "Chleb_na_zakwasie": 200
}

# Variables for production, sales, and storage of products each day
sub_production_vars = pl.LpVariable.dicts("Subproduct production", ((subproduct, day) for subproduct in subproducts for day in days), lowBound=0, cat='Integer')
production_vars = pl.LpVariable.dicts("Production", ((product, day) for product in products for day in days), lowBound=0, cat='Integer')
# batch_count_vars = pl.LpVariable.dicts("Batches Count", ((product, day) for product in products for day in days), lowBound=0, cat='Integer')
# subbatch_count_vars = pl.LpVariable.dicts("Subproduct Batches Count", ((subproduct, day) for subproduct in subproducts for day in days), lowBound=0, cat='Integer')
product_batch_start_time = pl.LpVariable.dicts("ProductStartTime",  [(product, machine, day) for product in products for machine in machines for day in days], lowBound=0, cat="Continuous")
subproduct_batch_start_time = pl.LpVariable.dicts("Subproduction start time for batch",  [(subproduct, machine, day) for subproduct in subproducts for machine in machines for day in days], lowBound=0, cat="Continuous")
sales_vars = pl.LpVariable.dicts("Sales", ((product, day) for product in products for day in days), lowBound=0, cat='Integer')
# specific_machine_used = pl.LpVariable.dicts("SpecificMachineUsed", [(machine, idx) for (machine, idx) in specific_machines], cat="Binary")
# subbatch_produced = pl.LpVariable.dicts("BatchProducedSubproduct", [(subproduct, day, batch) for subproduct in subproducts for day in days for batch in range(1, max_batches + 1)], cat="Binary")
subbatch_produced = pl.LpVariable.dicts("BatchProducedSubproduct", [(subproduct, day) for subproduct in subproducts for day in days], cat="Binary")
batch_produced = pl.LpVariable.dicts("BatchProduced", [(product, day, batch) for product in products for day in days for batch in range(1, max_batches + 1)], cat="Binary")
specific_machine_used = pl.LpVariable.dicts(
    "SpecificMachineUsed",
    [(machine, subproduct, day) for machine in machines for subproduct in subproducts for day in days],
    cat="Binary"
)
# xij = pl.LpVariable.dicts("xij", [(machine, idx, i, j) for (machine, idx) in specific_machines for i in subproducts for j in subproducts if i != j], 0, 1, pl.LpBinary)
is_produced = pl.LpVariable.dicts("is_produced",
                                     ((subproduct, machine, day) for subproduct in subproducts
                                                                      for machine in machines
                                                                      for day in days),
                                     cat="Binary")

for subproduct in subproducts:
    for day in days:
        for i, machine in enumerate(machines[1:], start=1):  # Od ważenia do nadziewarki
            prev = 1
            prev_machine = machines[i - prev]

            while production_times[subproduct][prev_machine] == 0.0 or prev_machine == machine:
                prev += 1
                prev_machine = machines[i - prev]

            model += subproduct_batch_start_time[subproduct, machine, day] >= (
                subproduct_batch_start_time[subproduct, prev_machine, day] + production_times[subproduct][prev_machine]
            )

            model += subproduct_batch_start_time[subproduct, machine, day] <= specific_machine_used[machine, subproduct, day] * hours_per_day

        last_machine = machines[-1]
        model += hours_per_day >= (
            subproduct_batch_start_time[subproduct, last_machine, day] + production_times[subproduct][last_machine]
        )
'''
        model += subbatch_produced[subproduct, day] * large_number >= subproduct_batch_start_time[subproduct, last_machine, last_idx, day]

        # Brak nakładania się produkcji na maszynie
        for machine, idx in specific_machines[:6]:  # Zakres maszyn
            for subproduct1 in subproducts:
                for subproduct2 in subproducts:
                    if subproduct1 != subproduct2:
                        model += (subproduct_batch_start_time[subproduct1, machine, idx, day]
                                  >= subproduct_batch_start_time[subproduct2, machine, idx, day] + production_times[subproduct2][machine]) | \
                                 (subproduct_batch_start_time[subproduct2, machine, idx, day]
                                  >= subproduct_batch_start_time[subproduct1, machine, idx, day] + production_times[subproduct1][machine])

                        # model += specific_machine_used[machine, idx, subproduct1, day] + specific_machine_used[machine, idx, subproduct2, day] <= 1
'''
# Ograniczenie: produkcja subproduktu tylko na maszynach, gdzie czas produkcji > 0
for subproduct in subproducts:
    for machine in machines:
        for day in days:
            if production_times[subproduct][machine] == 0:
                model += is_produced[subproduct, machine, day] == 0
                

# Warunek: jeśli subprodukt jest produkowany, to na wszystkich maszynach (z czasem > 0)
for subproduct in subproducts:
    machines_with_production = [machine for machine in machines if production_times[subproduct][machine] > 0]
    for day in days:
        if machines_with_production:
            # Ograniczenie wymuszające produkcję na wszystkich maszynach albo na żadnej
            for machine in machines_with_production:
                    model += pl.lpSum(is_produced[subproduct, m, day] for m in machines_with_production) == \
                            len(machines_with_production) * is_produced[subproduct, machine, day]
        
        # pozowolenie subbatch_produced na wartość 1 jeżeli na maszyny działały dla danego wyrobu
        model += subbatch_produced[subproduct, day] <= pl.lpSum(is_produced[subproduct, machine, day] for machine in machines_with_production)

# Aktualizacja ograniczenia: brak nakładania się produkcji
big_M = 1000000
for subproduct1 in subproducts:
    for subproduct2 in subproducts:
        if subproduct1 != subproduct2:
            machines_with_production = [machine for machine in machines if production_times[subproduct1][machine] > 0 and production_times[subproduct2][machine] > 0]
            for machine in machines_with_production: 
                for day in days:
                    # Dodanie binarnych zmiennych pomocniczych
                    y1 = pl.LpVariable(f"y1_{subproduct1}_{subproduct2}_{machine}_{day}", cat="Binary")
                    y2 = pl.LpVariable(f"y2_{subproduct1}_{subproduct2}_{machine}_{day}", cat="Binary")
                    y3 = pl.LpVariable(f"y3_{subproduct1}_{subproduct2}_{machine}_{day}", cat="Binary")
                    y4 = pl.LpVariable(f"y4_{subproduct1}_{subproduct2}_{machine}_{day}", cat="Binary")

                    # Powiązanie warunków logicznych z binarnymi zmiennymi
                    model += subproduct_batch_start_time[subproduct1, machine, day] >= \
                                subproduct_batch_start_time[subproduct2, machine, day] + production_times[subproduct2][machine] - \
                                big_M * (1 - y1)
                    
                    model += subproduct_batch_start_time[subproduct2, machine, day] >= \
                                subproduct_batch_start_time[subproduct1, machine, day] + production_times[subproduct1][machine] - \
                                big_M * (1 - y2)

                    model += is_produced[subproduct1, machine, day] + y3 == 1
                    model += is_produced[subproduct2, machine, day] + y4 == 1

                    # Zapewnienie, że przynajmniej jedno ograniczenie jest spełnione
                    model += y1 + y2 + y3 + y4 == 1



for product, subproduct in zip(products, subproducts):
    for day in days:
        model += subbatch_produced[subproduct, day] * batch_size[subproduct] >= sub_production_vars[subproduct, day]
        model += sub_production_vars[subproduct, day] >= production_vars[product, day]
        model += sales_vars[product, day] <= production_vars[product, day]


# Objective function: Maximize profit from sales minus costs
model += pl.lpSum([sales_vars[product, day] * product_income[product] for product in products for day in days]) 
                    # - pl.lpSum([production_vars[product, day] * product_cost[product] for product in products for day in days]) \
                    # - pl.lpSum([sub_production_vars[subproduct, day] * sub_product_cost[subproduct] for subproduct in subproducts for day in days])


solver = pl.PULP_CBC_CMD(timeLimit=20)

model.solve(solver)

# model.solve()

# Results
print("Status:", pl.LpStatus[model.status])
if pl.LpStatus[model.status] == 'Optimal':
    print("Total Profit:", pl.value(model.objective))

    print(f"{'Product':<10} {'day':<10} {'Production':<12} {'Sales':<10} {'Storage':<10}")

    for product, subproduct in zip(products, subproducts):
        for day in days:
            subproduction = sub_production_vars[subproduct, day].varValue
            production = production_vars[product, day].varValue
            sales = sales_vars[product, day].varValue

            print(f"{product:<10} {day:<10} {subproduction:<12} {production:<12} {sales:<10}")

    # print("\nProduction Time Usage per Product and Day:")
    # for day in days:
    #     print(f"\Day: {day}")
    #     for product in products:
    #         total_production_time = sum(
    #             batch_count_vars[product, day].varValue * production_times[product]s.get(machine, 0) 
    #             for machine in machines #.keys()
    #         )
    #         print(f"  Product: {product}, Total Production Time: {total_production_time:.2f} hours")

else:
    print("No optimal solution was found.")
    # Analiza ograniczeń
    for name, constraint in model.constraints.items():
        if constraint.pi < 0:
            print(f"Ograniczenie {name}: {constraint}")

Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /home/gromiusz/.local/lib/python3.10/site-packages/pulp/solverdir/cbc/linux/64/cbc /tmp/8b3b5f849a254e5d80dd2ebc39156282-pulp.mps -max -sec 20 -timeMode elapsed -branch -printingOptions all -solution /tmp/8b3b5f849a254e5d80dd2ebc39156282-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 131 COLUMNS
At line 669 RHS
At line 796 BOUNDS
At line 896 ENDATA
Problem MODEL has 126 rows, 108 columns and 336 elements
Coin0008I MODEL read with 0 errors
seconds was changed from 1e+100 to 20
Option for timeMode changed from cpu to elapsed
Continuous objective value is 3515 - 0.00 seconds
Cgl0003I 0 fixed, 0 tightened bounds, 39 strengthened rows, 30 substitutions
Cgl0003I 0 fixed, 0 tightened bounds, 32 strengthened rows, 0 substitutions
Cgl0003I 0 fixed, 0 tightened bounds, 29 strengthened rows, 0 substitutions
Cgl0004I processed model has 63 rows, 54 columns (45 integer (36 o